In [1]:
%load_ext autoreload
%autoreload 2

from typing import List, Tuple
import torch

import pytorch_lightning as pl
from src.models.model import RsnaTimmModel
from src.data.datamodule import RsnaDataModule


from pytorch_lightning.utilities.seed import seed_everything
seed_everything(44, True)


RUN_NAME = ""
DESC = ""
MODEL_NAME = "tf_efficientnetv2_s"


conf = {
    "dataset_root": "/data/rsna/",
    "augments_name": "SimpleAugments",
    "augments_kwargs": {"img_size": 1024},
    "val_ratio": 0.2,
    "img_size": 1024,
    "img_folder": "/data/rsna/train_images_png_1024",
    "img_folder_inference": "/data/rsna/test_images_1024/",
    "batch_size": 8,
    "num_workers": 16,
    "model_name": MODEL_NAME,
    'pretrained': False,
    "fc_dropout": 0.65,
    "lr": 5e-5,
    "category_aux_targets": [
        "site_id",
        "laterality",
        "view",
        "implant",
        "biopsy",
        "invasive",
        "BIRADS",
        "density",
        "difficult_negative_case",
        "machine_id",
        "age",
    ],
    "aux_classes": [2, 2, 6, 2, 2, 2, 4, 5, 2, 10, 10],
    "aux_loss_weight": 1.0,  # ?
}

/home/toomuch/anaconda3/envs/torch/lib/python3.10/site-packages/nvidia/dali/backend.py:46: Warning: DALI support for Python 3.10 is experimental and some functionalities may not work.
  deprecation_warning("DALI support for Python 3.10 is experimental and some functionalities "
/home/toomuch/anaconda3/envs/torch/lib/python3.10/site-packages/pytorch_lightning/utilities/seed.py:48: LightningDeprecationWarning: `pytorch_lightning.utilities.seed.seed_everything` has been deprecated in v1.8.0 and will be removed in v1.10.0. Please use `lightning_lite.utilities.seed.seed_everything` instead.
  rank_zero_deprecation(
Global seed set to 44


In [2]:
dm = RsnaDataModule(conf)
dm.prepare_data()
dm.setup()

model = RsnaTimmModel(conf)#.load_from_checkpoint('/home/toomuch/rsna/runs/[READY 0.294] EffNet2_s_1024_AUX/tf_efficientnetv2_s_1024_debug-epoch=09-val_loss=2.238-val_cf1=0.062_cf1thr=0.3.ckpt')
print(model.load_state_dict(torch.load('/home/toomuch/rsna/runs/[READY 0.294] EffNet2_s_1024_AUX/tf_efficientnetv2_s_1024_debug-epoch=09-val_loss=2.238-val_cf1=0.062_cf1thr=0.3.ckpt')['state_dict']))

/home/toomuch/anaconda3/envs/torch/lib/python3.10/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


<All keys matched successfully>


In [3]:
trainer = pl.Trainer(accelerator='gpu', devices=1, precision=16)
predictions = trainer.predict(model, dm)

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting: 0it [00:00, ?it/s]

In [4]:
# trainer.validate(model, dm)
#===========================================================================#
            # = VALID LOSS: 2.076
            # = VALID cF1: 0.074
            # = VALID cF1_thresh 0.486, thr value = 0.846
#===========================================================================#

In [5]:
result = []
for batch in predictions:
    cancer_probs, patient_ids, lateralitys = batch
    cancer_probs = cancer_probs.detach().cpu().numpy()
    for prob, patient_id, laterality in zip(cancer_probs, patient_ids, lateralitys):
        prediction_id = '{}_{}'.format(patient_id, laterality)
        result.append([prediction_id, float(prob)])

In [6]:
import pandas as pd
THRESHOLD = 0.846

df = pd.DataFrame(result, columns = ['prediction_id', 'cancer'])
sub = df[['prediction_id', 'cancer']].groupby("prediction_id").mean().reset_index()
sub["cancer"] = (sub["cancer"] > THRESHOLD).astype(int)
sub

,prediction_id,cancer
0,10008_L,0
1,10008_R,0
